In [1]:
import os

In [2]:
pip install transformers==4.51.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.3/509.3 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 75.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.28.1
    Uninstalling huggingface-hub-0.28.1:
      Successfully uninstalled huggingface-hub-0.28.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


In [11]:
pip install -q peft torchinfo dagshub mlflow datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.5 MB/s eta 0:00:00


In [4]:
import transformers

print(transformers.__version__)

4.51.3


In [5]:
import dagshub

dad = dagshub.init(
    repo_owner="ashish.student2025",
    repo_name="AirTravel_SentimentAnalysis",
    mlflow=True,
)
import mlflow

mlflow.set_tracking_uri(
    "https://dagshub.com/ashish.student2025/AirTravel_SentimentAnalysis.mlflow"
)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=30831114-e0df-4ff0-bd5a-1050b2f9d024&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=24db4b785b2bfe3019ff509f89d539b53a3dc5a45f01ef533b55c3e3cd165e10




Accessing as ashish.student2025

Initialized MLflow to track repo "ashish.student2025/AirTravel_SentimentAnalysis"

Repository ashish.student2025/AirTravel_SentimentAnalysis initialized!

In [6]:
os.getenv("MLFLOW_TRACKING_URI").split(os.sep)[-2]

'ashish.student2025'

In [36]:
mlflow.set_experiment("AirTravel_SentimentAnalysis-baseline-peft-it9")

2025/05/24 16:58:29 INFO mlflow.tracking.fluent: Experiment with name 'AirTravel_SentimentAnalysis-baseline-peft-it9' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/24239bc481d6442cae1234ea493c88aa', creation_time=1748105909488, experiment_id='10', last_update_time=1748105909488, lifecycle_stage='active', name='AirTravel_SentimentAnalysis-baseline-peft-it9', tags={}>

In [8]:
from google.colab import drive
d= drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
test_file_path = "/content/drive/MyDrive/test"
train_file_path = "/content/drive/MyDrive/train"
val_file_path = "/content/drive/MyDrive/val"

In [37]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
from transformers import TrainingArguments
from transformers import Trainer
from transformers import AutoTokenizer
from transformers import pipeline
from transformers import AutoModelForSequenceClassification
from datasets import load_from_disk
from torchinfo import summary
from peft import LoraConfig, TaskType
from peft import get_peft_model
import mlflow
import os

mlflow.end_run() # End any active run

def compute_metrics(eval_pred):
    print(
        "*******************************************************************compute_metrics called +++++++++++++++++++++++++++++++++++++++++++++++"
    )
    try:
        logits, labels = eval_pred
        print(f"Logits shape: {logits.shape}, Labels shape: {labels.shape}")
        predictions = np.argmax(
            logits, axis=-1
        )  # Predicted class is the index of max logit
        print(f"Predictions shape: {predictions.shape}")
        print(f"Unique labels: {np.unique(labels)}")
        print(f"Unique predictions: {np.unique(predictions)}")

        precision = precision_score(
            labels, predictions, average="weighted", zero_division=0
        )
        recall = recall_score(labels, predictions, average="weighted", zero_division=0)
        f1 = f1_score(labels, predictions, average="weighted", zero_division=0)
        accuracy = accuracy_score(labels, predictions)

        metrics = {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1,
        }
        print(f"Computed metrics: {metrics}")
        return metrics
    except Exception as e:
        print(f"Error in compute_metrics: {e}")
        return {}


training_args = TrainingArguments(
    output_dir="artifacts/model_training/finetuned_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,  # Learning rate was 2e-5
    per_device_train_batch_size=8,  # Batch size for training
    per_device_eval_batch_size=8,  # Batch size for evaluation
    num_train_epochs=1,  # Number of training epochs
    weight_decay=0.01,  # Weight decay for regularization
    load_best_model_at_end=True,  # Load best model after training
    metric_for_best_model="f1",
)
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=1,
    lora_alpha=1,
    lora_dropout=0.1,
    target_modules=["q_lin", "v_lin"],
)
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=33 # Explicitly set the number of labels
)
print("*******************without peft*******************")
print(summary(model))
print("*******************Model*******************")
print(model)
print("*******************with peft*******************")
peft_model = get_peft_model(model, lora_config)
print(summary(peft_model))
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenized_train_dataset = load_from_disk(train_file_path) # Use variable
tokenized_train_dataset = tokenized_train_dataset.rename_column("intent", "labels")
tokenized_test_dataset = load_from_disk(test_file_path) # Use variable
tokenized_test_dataset = tokenized_test_dataset.rename_column("intent", "labels")
tokenized_val_dataset = load_from_disk(val_file_path) # Use variable
tokenized_val_dataset = tokenized_val_dataset.rename_column("intent", "labels")
# print("=== TRAIN DATASET SAMPLING ===")
# tokenized_train_dataset = get_balanced_subset(tokenized_train_dataset, total_rows=1000)

# print("\n=== TEST DATASET SAMPLING ===")
# tokenized_test_dataset = get_balanced_subset(tokenized_test_dataset, total_rows=200)

# print("\n=== VAL DATASET SAMPLING ===")
# tokenized_val_dataset = get_balanced_subset(tokenized_val_dataset, total_rows=200)
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
with mlflow.start_run(run_name="baseline-peft-it9") as run:
    trainer.train()
    metrics = trainer.evaluate()
    mlflow.log_metrics(metrics)
trainer.save_model("artifacts/model_training/mlfow_model")
tokenizer.save_pretrained("artifacts/model_training/tokenizer")
tuned_pipeline = pipeline(
    task="text-classification",
    model=trainer.model,
    batch_size=8,
    tokenizer=tokenizer,
    device="cpu",
)
model_config = {"batch_size": 8}
model_info = mlflow.transformers.log_model(
    transformers_model=tuned_pipeline,
    artifact_path="fine_tuned",
    model_config=model_config,
)
# mlflow.transformers.persist_pretrained_model(model_info.model_uri)
print("Model saved in run %s" % model_info.model_uri)
loaded = mlflow.transformers.load_model(model_uri=model_info.model_uri)

decoded_texts = ()
for i in range(len(tokenized_val_dataset)):
    decoded_text = tokenizer.decode(
        tokenized_val_dataset[i]["input_ids"],
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True,
    )
    b = list(decoded_texts)
    b.append(decoded_text)
    decoded_texts = tuple(b)
loaded(list(tokenized_val_dataset))
mlflow.end_run()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


*******************without peft*******************
Layer (type:depth-idx)                                  Param #
DistilBertForSequenceClassification                     --
├─DistilBertModel: 1-1                                  --
│    └─Embeddings: 2-1                                  --
│    │    └─Embedding: 3-1                              23,440,896
│    │    └─Embedding: 3-2                              393,216
│    │    └─LayerNorm: 3-3                              1,536
│    │    └─Dropout: 3-4                                --
│    └─Transformer: 2-2                                 --
│    │    └─ModuleList: 3-5                             42,527,232
├─Linear: 1-2                                           590,592
├─Linear: 1-3                                           25,377
├─Dropout: 1-4                                          --
Total params: 66,978,849
Trainable params: 66,978,849
Non-trainable params: 0
*******************Model*******************
DistilBertForSequenceC

<ipython-input-37-a71abcb03b9b>:95: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
2025/05/24 16:59:00 ERROR mlflow.utils.async_logging.async_logging_queue: Run Id 18ea58c83df1484a9c40e058f483f3ea: Failed to log run data: Exception: BAD_REQUEST: Response: {'error_code': 'BAD_REQUEST'}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.567500,2.458624,0.531780,0.599608,0.531780,0.453282


2025/05/24 16:59:00 ERROR mlflow.utils.async_logging.async_logging_queue: Run Id 18ea58c83df1484a9c40e058f483f3ea: Failed to log run data: Exception: BAD_REQUEST: Response: {'error_code': 'BAD_REQUEST'}


*******************************************************************compute_metrics called +++++++++++++++++++++++++++++++++++++++++++++++
Logits shape: (5066, 33), Labels shape: (5066,)
Predictions shape: (5066,)
Unique labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32]
Unique predictions: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 20 21 22 24 25
 26 27 28 29 30 31]
Computed metrics: {'accuracy': 0.5317804974338729, 'precision': 0.5996079864875173, 'recall': 0.5317804974338729, 'f1': 0.45328184072932926}


*******************************************************************compute_metrics called +++++++++++++++++++++++++++++++++++++++++++++++
Logits shape: (5066, 33), Labels shape: (5066,)
Predictions shape: (5066,)
Unique labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32]
Unique predictions: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 20 21 22 24 25
 26 27 28 29 30 31]
Computed metrics: {'accuracy': 0.5317804974338729, 'precision': 0.5996079864875173, 'recall': 0.5317804974338729, 'f1': 0.45328184072932926}
🏃 View run baseline-peft-it9 at: https://dagshub.com/ashish.student2025/AirTravel_SentimentAnalysis.mlflow/#/experiments/10/runs/18ea58c83df1484a9c40e058f483f3ea
🧪 View experiment at: https://dagshub.com/ashish.student2025/AirTravel_SentimentAnalysis.mlflow/#/experiments/10


Device set to use cpu
The model 'PeftModelForSequenceClassification' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DiffLlamaForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'Funnel

2025/05/24 17:14:18 WARNING mlflow.transformers.model_io: Could not specify device parameter for this pipeline type.Falling back to loading the model with the default device.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: Error(s) in loading state_dict for PeftModelForSequenceClassification:
	size mismatch for base_model.model.classifier.modules_to_save.default.weight: copying a param with shape torch.Size([33, 768]) from checkpoint, the shape in current model is torch.Size([2, 768]).
	size mismatch for base_model.model.classifier.modules_to_save.default.bias: copying a param with shape torch.Size([33]) from checkpoint, the shape in current model is torch.Size([2]).

In [40]:
mlflow.transformers.log_model(
        transformers_model=tuned_pipeline, artifact_path="pipeline", save_pretrained=False
    )

2025/05/24 17:23:38 INFO mlflow.transformers: Overriding save_pretrained to False for PEFT models, following the Transformers behavior. The PEFT adaptor and config will be saved, but the base model weights will not and reference to the HuggingFace Hub repository will be logged instead.
2025/05/24 17:23:46 INFO mlflow.transformers: Skipping saving pretrained model weights to disk as the save_pretrained argumentis set to False. The reference to the HuggingFace Hub repository distilbert-base-uncased will be logged instead.
2025/05/24 17:23:49 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.1+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torch==2.5.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/05/24 17:23:49 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.20.1+cu1

In [41]:
mlflow.transformers.persist_pretrained_model("runs:/dedee728d5d645da90fc0082ca07c1fb/pipeline")

2025/05/24 17:25:12 WARNING mlflow.transformers.model_io: Could not specify device parameter for this pipeline type.Falling back to loading the model with the default device.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: Error(s) in loading state_dict for PeftModelForSequenceClassification:
	size mismatch for base_model.model.classifier.modules_to_save.default.weight: copying a param with shape torch.Size([33, 768]) from checkpoint, the shape in current model is torch.Size([2, 768]).
	size mismatch for base_model.model.classifier.modules_to_save.default.bias: copying a param with shape torch.Size([33]) from checkpoint, the shape in current model is torch.Size([2]).

In [88]:
import shutil

# Save the model to a temporary directory first
temp_output_dir = "/tmp/model_save"
trainer.save_model(temp_output_dir)

# Then copy the saved model files to the desired Google Drive location
drive_save_path = "/content/drive/MyDrive/model"

# Create the target directory in Google Drive if it doesn't exist
os.makedirs(drive_save_path, exist_ok=True)

# Copy the contents of the temporary directory to the Google Drive directory
shutil.copytree(temp_output_dir, drive_save_path, dirs_exist_ok=True)

print(f"Model successfully saved to {drive_save_path}")

Model successfully saved to /content/drive/MyDrive/model


In [74]:
temp_output_dir = "/tmp/tokenizer_save"
tokenizer.save_pretrained(temp_output_dir)

# Then copy the saved model files to the desired Google Drive location
drive_save_path = "/content/drive/MyDrive/tokenizer"

# Create the target directory in Google Drive if it doesn't exist
os.makedirs(drive_save_path, exist_ok=True)

# Copy the contents of the temporary directory to the Google Drive directory
shutil.copytree(temp_output_dir, drive_save_path, dirs_exist_ok=True)

print(f"Model successfully saved to {drive_save_path}")

Model successfully saved to /content/drive/MyDrive/tokenizer


In [18]:
mlflow.end_run()

🏃 View run languid-doe-13 at: https://dagshub.com/ashish.student2025/AirTravel_SentimentAnalysis.mlflow/#/experiments/9/runs/5688084717c94778ad57fd95e27c56ee
🧪 View experiment at: https://dagshub.com/ashish.student2025/AirTravel_SentimentAnalysis.mlflow/#/experiments/9


In [24]:
from peft import PeftConfig
peft_config = PeftConfig.from_pretrained("artifacts/model_training/mlfow_model")


In [25]:
peft_config

LoraConfig(task_type='SEQ_CLS', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='distilbert-base-uncased', revision=None, inference_mode=True, r=1, target_modules={'q_lin', 'v_lin'}, exclude_modules=None, lora_alpha=1, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=['classifier', 'score'], init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, eva_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

In [20]:
import mlflow
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel

# Load your saved model
base_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=33)
peft_model = PeftModel.from_pretrained(base_model, "artifacts/model_training/mlfow_model")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Create inference pipeline
pipe = pipeline("text-classification", model=peft_model, tokenizer=tokenizer)

# Log the model to MLflow
with mlflow.start_run():
    mlflow.transformers.log_model(
        transformers_model=pipe,
        artifact_path="peft_classifier",
        input_example=["The flight was fantastic!", "It was delayed and horrible."],
    )
    print("Model logged to MLflow")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
The model 'PeftModelForSequenceClassification' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'Deberta

Model logged to MLflow
🏃 View run magnificent-cow-739 at: https://dagshub.com/ashish.student2025/AirTravel_SentimentAnalysis.mlflow/#/experiments/9/runs/ff40495f50fd42bf8fd88c4da3215863
🧪 View experiment at: https://dagshub.com/ashish.student2025/AirTravel_SentimentAnalysis.mlflow/#/experiments/9


In [21]:
import mlflow

# Replace with your actual run_id or use the one just printed
model_uri = "runs:/ff40495f50fd42bf8fd88c4da3215863/peft_classifier"
model = mlflow.transformers.load_model(model_uri)


2025/05/24 16:36:52 INFO mlflow.transformers: 'runs:/ff40495f50fd42bf8fd88c4da3215863/peft_classifier' resolved as 'mlflow-artifacts:/1fce69cd893c4735982c7075a799f5ac/ff40495f50fd42bf8fd88c4da3215863/artifacts/peft_classifier'


2025/05/24 16:36:54 WARNING mlflow.transformers.model_io: Could not specify device parameter for this pipeline type.Falling back to loading the model with the default device.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: Error(s) in loading state_dict for PeftModelForSequenceClassification:
	size mismatch for base_model.model.classifier.modules_to_save.default.weight: copying a param with shape torch.Size([33, 768]) from checkpoint, the shape in current model is torch.Size([2, 768]).
	size mismatch for base_model.model.classifier.modules_to_save.default.bias: copying a param with shape torch.Size([33]) from checkpoint, the shape in current model is torch.Size([2]).

In [26]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from peft import PeftModel, PeftConfig

# Load PEFT config from disk
peft_config = PeftConfig.from_pretrained("artifacts/model_training/mlfow_model")

# Load the base model with correct num_labels
base_model = AutoModelForSequenceClassification.from_pretrained(
    peft_config.base_model_name_or_path,
    num_labels=33  # Set to match your training
)

# Load the PEFT adapter on top of the base model
model = PeftModel.from_pretrained(base_model, "artifacts/model_training/mlfow_model")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("artifacts/model_training/tokenizer")

# Use for inference
clf = pipeline("text-classification", model=model, tokenizer=tokenizer)
print(clf("This flight was delayed but the service was nice."))

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
The model 'PeftModelForSequenceClassification' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'Deberta

[{'label': 'LABEL_13', 'score': 0.06516291946172714}]


In [31]:
decoded_texts = ()
for i in range(len(tokenized_val_dataset)):
    decoded_text = tokenizer.decode(
        tokenized_val_dataset[i]["input_ids"],
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True,
    )
    b = list(decoded_texts)
    b.append(decoded_text)
    decoded_texts = tuple(b)
clf(list(decoded_texts))

[{'label': 'LABEL_5', 'score': 0.10764039307832718},
 {'label': 'LABEL_13', 'score': 0.11496108025312424},
 {'label': 'LABEL_21', 'score': 0.1783648580312729},
 {'label': 'LABEL_21', 'score': 0.07133596390485764},
 {'label': 'LABEL_13', 'score': 0.11538542062044144},
 {'label': 'LABEL_21', 'score': 0.08573829382658005},
 {'label': 'LABEL_20', 'score': 0.0975806787610054},
 {'label': 'LABEL_17', 'score': 0.0774955302476883},
 {'label': 'LABEL_21', 'score': 0.16841547191143036},
 {'label': 'LABEL_20', 'score': 0.08139245212078094},
 {'label': 'LABEL_8', 'score': 0.2142079919576645},
 {'label': 'LABEL_15', 'score': 0.11379926651716232},
 {'label': 'LABEL_9', 'score': 0.0948714166879654},
 {'label': 'LABEL_15', 'score': 0.13501684367656708},
 {'label': 'LABEL_26', 'score': 0.13393093645572662},
 {'label': 'LABEL_16', 'score': 0.0765506774187088},
 {'label': 'LABEL_16', 'score': 0.07230130583047867},
 {'label': 'LABEL_15', 'score': 0.09938091039657593},
 {'label': 'LABEL_32', 'score': 0.078

In [32]:
tokenized_val_dataset

Dataset({
    features: ['instruction', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 6332
})

In [34]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): DistilBertSdpaAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=76

In [35]:
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

dataloader = DataLoader(tokenized_val_dataset, batch_size=8)

all_preds = []
all_labels = []

for batch in dataloader:
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, axis=-1)

    all_preds.extend(preds.cpu().numpy())
    all_labels.extend(labels.cpu().numpy())

# Compute metrics
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average="weighted")
recall = recall_score(all_labels, all_preds, average="weighted")
f1 = f1_score(all_labels, all_preds, average="weighted")

print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

AttributeError: 'list' object has no attribute 'to'